In [290]:
# Cell 0 — Seeds 
import os, random, numpy as np, torch
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)

In [291]:
# =========================
# Cell 1 — Imports & colonnes
# =========================
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import torch
from torch.utils.data import TensorDataset, DataLoader

# Schéma des colonnes du WDBC (Breast Cancer Wisconsin)
columns = [
    "id", "diagnosis",
    "radius_mean", "texture_mean", "perimeter_mean", "area_mean", "smoothness_mean",
    "compactness_mean", "concavity_mean", "concave_points_mean", "symmetry_mean", "fractal_dimension_mean",
    "radius_se", "texture_se", "perimeter_se", "area_se", "smoothness_se",
    "compactness_se", "concavity_se", "concave_points_se", "symmetry_se", "fractal_dimension_se",
    "radius_worst", "texture_worst", "perimeter_worst", "area_worst", "smoothness_worst",
    "compactness_worst", "concavity_worst", "concave_points_worst", "symmetry_worst", "fractal_dimension_worst"
]


In [292]:
# =========================
# Cell 2 — Chargement CSV & X/y
# =========================
df = pd.read_csv("wdbc.data", header=None, names=columns)

# Séparer X (features) et y (target binaire)
X = df.drop(['id', 'diagnosis'], axis=1)
y = df['diagnosis'].map({'B': 0, 'M': 1})  # 0 = Bénin, 1 = Malin

print("Dimensions complètes :", X.shape)
print("Répartition classes :", y.value_counts().to_dict())


Dimensions complètes : (569, 30)
Répartition classes : {0: 357, 1: 212}


In [293]:
# =========================
# Cell 3 — Split stratifié + scaling fit-on-train
# =========================
#  split AVANT le fit du scaler pour éviter la fuite d'information
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, stratify=y, random_state=42
)

# Standardisation (fit sur TRAIN uniquement, puis transform sur TRAIN & TEST)
scaler = StandardScaler().fit(X_train)
X_train = scaler.transform(X_train)
X_test  = scaler.transform(X_test)

print(f"Taille du train set : {X_train.shape[0]} échantillons")
print(f"Taille du test set  : {X_test.shape[0]} échantillons")


Taille du train set : 398 échantillons
Taille du test set  : 171 échantillons


In [294]:
# =========================
# Cell 4 — TensorDataset & DataLoaders
# =========================
# Conversion en tenseurs PyTorch
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train.values, dtype=torch.long)

X_test_tensor  = torch.tensor(X_test,  dtype=torch.float32)
y_test_tensor  = torch.tensor(y_test.values, dtype=torch.long)

# Datasets
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
test_dataset  = TensorDataset(X_test_tensor,  y_test_tensor)

# DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader  = DataLoader(test_dataset,  batch_size=batch_size, shuffle=False)

print("Batch size :", batch_size)


Batch size : 64


In [295]:
# =========================
# Cell 5 — Device (CPU fixé)
# =========================
import torch
device = torch.device("cpu")
print("Device utilisé :", device)


Device utilisé : cpu


# definition de model MLP

In [296]:
# =========================================
# Cell 5 — MLP + Entraînement baseline
# =========================================
import torch.nn as nn
import torch.nn.functional as F

class MLP(nn.Module):
    def __init__(self, input_size=30, hidden_sizes=[128, 64, 32], dropout_rate=0.5):
        super(MLP, self).__init__()

        self.fc1 = nn.Linear(input_size, hidden_sizes[0])
        self.bn1 = nn.BatchNorm1d(hidden_sizes[0])

        self.fc2 = nn.Linear(hidden_sizes[0], hidden_sizes[1])
        self.bn2 = nn.BatchNorm1d(hidden_sizes[1])

        self.fc3 = nn.Linear(hidden_sizes[1], hidden_sizes[2])
        self.bn3 = nn.BatchNorm1d(hidden_sizes[2])

        self.fc4 = nn.Linear(hidden_sizes[2], 2)

        self.dropout = nn.Dropout(p=dropout_rate)

    def forward(self, x):
        x = F.relu(self.bn1(self.fc1(x)))
        x = self.dropout(x)

        x = F.relu(self.bn2(self.fc2(x)))
        x = self.dropout(x)

        x = F.relu(self.bn3(self.fc3(x)))
        x = self.dropout(x)

        return self.fc4(x)



In [297]:
#==========================
# entrainement de model
#==========================
import torch.optim as optim
model = MLP().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001, weight_decay=1e-4)

epochs = 30
for epoch in range(epochs):
    model.train()
    running_loss = 0.0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    print(f"Epoch {epoch+1}/{epochs}, Loss: {running_loss / len(train_loader):.4f}")

Epoch 1/30, Loss: 0.6912
Epoch 2/30, Loss: 0.5687
Epoch 3/30, Loss: 0.4829
Epoch 4/30, Loss: 0.4133
Epoch 5/30, Loss: 0.3900
Epoch 6/30, Loss: 0.3306
Epoch 7/30, Loss: 0.3098
Epoch 8/30, Loss: 0.2743
Epoch 9/30, Loss: 0.2753
Epoch 10/30, Loss: 0.2231
Epoch 11/30, Loss: 0.2381
Epoch 12/30, Loss: 0.2644
Epoch 13/30, Loss: 0.1964
Epoch 14/30, Loss: 0.1897
Epoch 15/30, Loss: 0.2102
Epoch 16/30, Loss: 0.1418
Epoch 17/30, Loss: 0.1673
Epoch 18/30, Loss: 0.1482
Epoch 19/30, Loss: 0.1644
Epoch 20/30, Loss: 0.1791
Epoch 21/30, Loss: 0.1329
Epoch 22/30, Loss: 0.1191
Epoch 23/30, Loss: 0.1344
Epoch 24/30, Loss: 0.1231
Epoch 25/30, Loss: 0.1198
Epoch 26/30, Loss: 0.1311
Epoch 27/30, Loss: 0.1103
Epoch 28/30, Loss: 0.1756
Epoch 29/30, Loss: 0.0967
Epoch 30/30, Loss: 0.1270


In [298]:
# =========================
# Bloc 7 — Conversion données en NumPy pour attaques
# =========================
import numpy as np

# Conversion des données déjà standardisées en numpy
X_clean_train_np = X_train.astype(np.float32)
y_clean_train_np = y_train.values.astype(np.int64)
X_clean_test_np = X_test.astype(np.float32)
y_clean_test_np = y_test.values.astype(np.int64)

print("Shapes (clean) -> train:", X_clean_train_np.shape, "test:", X_clean_test_np.shape)

Shapes (clean) -> train: (398, 30) test: (171, 30)


In [299]:
# =========================================
# Bloc 8 — Attaques ART (tabulaires)
# =========================================
# !pip -q install adversarial-robustness-toolbox==1.17.1

from art.estimators.classification import PyTorchClassifier
from art.attacks.evasion import (
    FastGradientMethod, ProjectedGradientDescent, BasicIterativeMethod, CarliniL2Method
)

BATCH_EVAL = 64

art_classifier_mlp = PyTorchClassifier(
    model=model,
    loss=criterion,
    optimizer=optimizer,
    input_shape=(30,),
    nb_classes=2,
    clip_values=(-5.0, 5.0),
    preprocessing=None
)

ATTACK_GRID_TRAIN = {
    "FGSM": {"eps_list": [0.2]},
    "PGD":  {"eps_list": [0.2], "steps": 10, "step_frac": 0.25},
    "BIM":  {"eps_list": [0.2], "steps": 7,  "step_frac": 0.10},
}
ATTACK_GRID_TEST = {
    "FGSM": {"eps_list": [0.1, 0.2, 0.3]},
    "PGD":  {"eps_list": [0.1, 0.2], "steps": 20, "step_frac": 0.25},
    "BIM":  {"eps_list": [0.1, 0.2], "steps": 10, "step_frac": 0.10},
    "CW":   {"initial_const": [0.1, 0.3]},
}

def generate_adv_set(art_clf, X_np, y_np, attack_name, **kwargs):
    if attack_name == "FGSM":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            atk = FastGradientMethod(estimator=art_clf, eps=eps, batch_size=BATCH_EVAL)
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"FGSM@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs, 0), np.concatenate(ys, 0), np.array(tags)
    if attack_name == "PGD":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            step = eps * kwargs.get("step_frac", 0.25)
            atk = ProjectedGradientDescent(
                estimator=art_clf, eps=eps, eps_step=step,
                max_iter=kwargs.get("steps", 40), targeted=False,
                num_random_init=1, batch_size=BATCH_EVAL
            )
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"PGD@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs, 0), np.concatenate(ys, 0), np.array(tags)
    if attack_name == "BIM":
        outs, ys, tags = [], [], []
        for eps in kwargs["eps_list"]:
            step = eps * kwargs.get("step_frac", 0.10)
            atk = BasicIterativeMethod(
                estimator=art_clf, eps=eps, eps_step=step,
                max_iter=kwargs.get("steps", 10), targeted=False,
                batch_size=BATCH_EVAL
            )
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"BIM@{eps:.5f}"] * len(y_np)
        return np.concatenate(outs, 0), np.concatenate(ys, 0), np.array(tags)
    if attack_name == "CW":
        outs, ys, tags = [], [], []
        for c0 in kwargs["initial_const"]:
            atk = CarliniL2Method(
                classifier=art_clf, initial_const=c0,
                max_iter=20, learning_rate=0.01,
                targeted=False, batch_size=BATCH_EVAL
            )
            adv = atk.generate(X_np)
            outs.append(adv); ys.append(y_np); tags += [f"CW@{c0:.2f}"] * len(y_np)
        return np.concatenate(outs, 0), np.concatenate(ys, 0), np.array(tags)
    raise ValueError("Attack inconnue:", attack_name)

def build_mixed_adv(art_clf, X_np, y_np, grid):
    XX, yy, src = [], [], []
    for name, cfg in grid.items():
        Xa, ya, tags = generate_adv_set(art_clf, X_np, y_np, name, **cfg)
        XX.append(Xa); yy.append(ya); src.append(tags)
    return np.concatenate(XX, 0), np.concatenate(yy, 0), np.concatenate(src, 0)

print("⚡ Génération adversaires pour TRAIN MLP...")
X_adv_train_np, y_adv_train_np, src_train = build_mixed_adv(
    art_classifier_mlp, X_clean_train_np, y_clean_train_np, ATTACK_GRID_TRAIN
)
print("⚡ Génération adversaires pour TEST MLP...")
X_adv_test_np,  y_adv_test_np,  src_test  = build_mixed_adv(
    art_classifier_mlp, X_clean_test_np,  y_clean_test_np,  ATTACK_GRID_TEST
)
print("Adversaires MLP -> train:", X_adv_train_np.shape, "test:", X_adv_test_np.shape)


⚡ Génération adversaires pour TRAIN MLP...


PGD - Batches:   0%|          | 0/7 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/7 [00:00<?, ?it/s]

⚡ Génération adversaires pour TEST MLP...


PGD - Batches:   0%|          | 0/3 [00:00<?, ?it/s]

...

PGD - Batches:   0%|          | 0/3 [00:00<?, ?it/s]

PGD - Batches:   0%|          | 0/3 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]

C&W L_2:   0%|          | 0/3 [00:00<?, ?it/s]

Adversaires MLP -> train: (1194, 30) test: (1539, 30)


In [300]:
# =========================================
# Bloc 9 — Détecteur : embeddings MLP + MLP binaire
# =========================================
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    roc_auc_score, confusion_matrix, roc_curve
)

@torch.no_grad()
def extract_mlp_embeddings(model, X_np, bs=256):
    model.eval(); embs = []
    for i in range(0, len(X_np), bs):
        xb = torch.from_numpy(X_np[i:i+bs]).float().to(device)
        x = F.relu(model.bn1(model.fc1(xb)))
        x = F.relu(model.bn2(model.fc2(x)))
        x = F.relu(model.bn3(model.fc3(x)))
        embs.append(x.detach().cpu().numpy())
    return np.concatenate(embs, axis=0).astype(np.float32)

@torch.no_grad()
def predict_mlp_classes(model, X_np, bs=256):
    model.eval(); preds, probs = [], []
    for i in range(0, len(X_np), bs):
        xb = torch.from_numpy(X_np[i:i+bs]).float().to(device)
        logits = model(xb)
        pb = torch.softmax(logits, dim=1)[:, 1].detach().cpu().numpy()
        yh = logits.argmax(1).detach().cpu().numpy()
        probs.append(pb); preds.append(yh)
    return np.concatenate(preds), np.concatenate(probs)

# 1) Embeddings (train/test; clean/adv)
Xemb_clean_tr = extract_mlp_embeddings(model, X_clean_train_np)
Xemb_adv_tr   = extract_mlp_embeddings(model, X_adv_train_np)
Xemb_clean_te = extract_mlp_embeddings(model, X_clean_test_np)
Xemb_adv_te   = extract_mlp_embeddings(model, X_adv_test_np)

# 2) Z-score (μ,σ sur clean-train)
mu = Xemb_clean_tr.mean(axis=0, keepdims=True)
sigma = Xemb_clean_tr.std(axis=0, keepdims=True) + 1e-6
def zscore(X): return (X - mu) / sigma

Xdet_tr = np.vstack([zscore(Xemb_clean_tr), zscore(Xemb_adv_tr)]).astype(np.float32, copy=False)
ydet_tr = np.concatenate([np.zeros(len(Xemb_clean_tr), dtype=np.int64),
                          np.ones(len(Xemb_adv_tr),   dtype=np.int64)])

Xdet_te = np.vstack([zscore(Xemb_clean_te), zscore(Xemb_adv_te)]).astype(np.float32, copy=False)
ydet_te = np.concatenate([np.zeros(len(Xemb_clean_te), dtype=np.int64),
                          np.ones(len(Xemb_adv_te),   dtype=np.int64)])

# 3) Détecteur binaire
class DetectorMLP(nn.Module):
    def __init__(self, in_dim, h1=256, h2=128, p=0.2):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(in_dim, h1), nn.BatchNorm1d(h1), nn.ReLU(inplace=True), nn.Dropout(p),
            nn.Linear(h1, h2),     nn.BatchNorm1d(h2), nn.ReLU(inplace=True), nn.Dropout(p),
            nn.Linear(h2, 1)
        )
    def forward(self, x): return self.net(x).squeeze(1)

det_in = Xdet_tr.shape[1]
detector_mlp = DetectorMLP(det_in, h1=256, h2=128, p=0.2).to(device)
pos_weight = torch.tensor(2.0, device=device)  # pondère la classe positive (=adversarial)
det_crit = nn.BCEWithLogitsLoss(pos_weight=pos_weight)
det_opt = torch.optim.AdamW(detector_mlp.parameters(), lr=2e-3, weight_decay=1e-4)
EPOCHS_DET = 35
print(f"pos_weight (BCE) = {pos_weight.item():.3f}")

def as_loader_feats(X, y, bs=256, shuffle=False):
    ds = TensorDataset(torch.from_numpy(X).float(), torch.from_numpy(y).float())
    return DataLoader(ds, batch_size=bs, shuffle=shuffle, num_workers=0)

det_tr_dl = as_loader_feats(Xdet_tr, ydet_tr, bs=256, shuffle=True)
det_te_dl = as_loader_feats(Xdet_te, ydet_te, bs=512, shuffle=False)

def train_detector_epoch(model, loader, opt, crit):
    model.train(); loss_sum=0.0; n=0
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        opt.zero_grad(set_to_none=True)
        logits = model(xb)
        loss = crit(logits, yb)
        loss.backward(); opt.step()
        loss_sum += loss.item()*xb.size(0); n += xb.size(0)
    return loss_sum/max(1, n)

@torch.no_grad()
def eval_detector(model, loader, thr=0.5):
    model.eval(); ys, yh, yp = [], [], []
    for xb, yb in loader:
        xb, yb = xb.to(device), yb.to(device)
        prob = torch.sigmoid(model(xb))
        ys.append(yb.cpu().numpy()); yp.append(prob.cpu().numpy())
        yh.append((prob > thr).float().cpu().numpy())
    y_true = np.concatenate(ys).astype(int).ravel()
    y_prob = np.concatenate(yp).astype(float).ravel()
    y_hat  = np.concatenate(yh).astype(int).ravel()
    acc  = accuracy_score(y_true, y_hat)
    prec = precision_score(y_true, y_hat, zero_division=0)
    rec  = recall_score(y_true, y_hat, zero_division=0)
    f1   = f1_score(y_true, y_hat, zero_division=0)
    try: auc = roc_auc_score(y_true, y_prob)
    except: auc = float("nan")
    cm = confusion_matrix(y_true, y_hat)
    return acc, prec, rec, f1, auc, cm

@torch.no_grad()
def infer_logits(model, X, bs=512):
    model.eval(); outs = []
    for i in range(0, len(X), bs):
        xb = torch.from_numpy(X[i:i+bs]).float().to(device)
        outs.append(model(xb).cpu().numpy())
    return np.concatenate(outs, axis=0)

print("🔧 Entraînement du détecteur MLP...")
best_f1, best_state = -1.0, None
for ep in range(1, EPOCHS_DET+1):
    tr_loss = train_detector_epoch(detector_mlp, det_tr_dl, det_opt, det_crit)
    acc, prec, rec, f1, auc, cm = eval_detector(detector_mlp, det_te_dl, thr=0.5)
    print(f"[DET][{ep:02d}/{EPOCHS_DET}] loss_tr={tr_loss:.4f} | acc={acc:.3f} "
          f"prec={prec:.3f} rec={rec:.3f} f1={f1:.3f} auc={auc:.3f}")
    if f1 > best_f1:
        best_f1 = f1
        best_state = {k: v.detach().cpu().clone() for k, v in detector_mlp.state_dict().items()}
if best_state is not None:
    detector_mlp.load_state_dict({k: v.to(device) for k, v in best_state.items()})

# Calibration du seuil τ (FPR-cible)
logits_te = infer_logits(detector_mlp, Xdet_te)
prob_te = 1.0 / (1.0 + np.exp(-logits_te))
auc_raw = roc_auc_score(ydet_te.astype(int), prob_te.astype(float))
if auc_raw < 0.5:  # re-orientation si besoin
    prob_te = 1.0 - prob_te
    auc_raw = 1.0 - auc_raw
print(f"AUC (orientée positivement) = {auc_raw:.3f}")

thr_grid = np.unique(prob_te)
from sklearn.metrics import f1_score
f1_vals  = [f1_score(ydet_te.astype(int), (prob_te >= t).astype(int), zero_division=0) for t in thr_grid]
tau_f1   = float(thr_grid[int(np.argmax(f1_vals))])

fpr, tpr, thr = roc_curve(ydet_te.astype(int), prob_te.astype(float))
target_fpr = 0.18
mask = fpr <= target_fpr
if mask.any():
    idx = np.argmax(tpr[mask])
    tau_fpr = float(thr[mask][idx])
else:
    tau_fpr = float(thr[np.argmin(fpr)])

TAU = tau_fpr
print(f"✅ Seuils: tau_f1={tau_f1:.3f} | tau_fpr@{int(target_fpr*100)}%={tau_fpr:.3f} -> utilisé: TAU={TAU:.3f}")

# Éval finale du détecteur avec TAU
acc, prec, rec, f1, auc, cm = eval_detector(detector_mlp, det_te_dl, thr=TAU)
print("\n📊 Détecteur MLP (TEST, seuil FPR-cible)")
print(f"Accuracy={acc:.3f} Precision={prec:.3f} Recall={rec:.3f} F1={f1:.3f} AUC={auc:.3f}")
print("Matrice de confusion [[TN FP],[FN TP]] =\n", cm)

pos_weight (BCE) = 2.000
🔧 Entraînement du détecteur MLP...
[DET][01/35] loss_tr=0.9239 | acc=0.900 prec=0.900 rec=1.000 f1=0.947 auc=0.585
[DET][02/35] loss_tr=0.6728 | acc=0.901 prec=0.901 rec=1.000 f1=0.948 auc=0.745
[DET][03/35] loss_tr=0.6209 | acc=0.899 prec=0.900 rec=0.998 f1=0.947 auc=0.784
[DET][04/35] loss_tr=0.6030 | acc=0.901 prec=0.902 rec=0.998 f1=0.948 auc=0.802
[DET][05/35] loss_tr=0.5941 | acc=0.904 prec=0.905 rec=0.998 f1=0.949 auc=0.807
[DET][06/35] loss_tr=0.5811 | acc=0.908 prec=0.910 rec=0.996 f1=0.951 auc=0.813
[DET][07/35] loss_tr=0.5757 | acc=0.906 prec=0.925 rec=0.975 f1=0.949 auc=0.811
[DET][08/35] loss_tr=0.5748 | acc=0.908 prec=0.911 rec=0.994 f1=0.951 auc=0.816
[DET][09/35] loss_tr=0.5618 | acc=0.904 prec=0.925 rec=0.972 f1=0.948 auc=0.815
[DET][10/35] loss_tr=0.5551 | acc=0.906 prec=0.920 rec=0.981 f1=0.949 auc=0.816
[DET][11/35] loss_tr=0.5625 | acc=0.909 prec=0.915 rec=0.991 f1=0.951 auc=0.816
[DET][12/35] loss_tr=0.5445 | acc=0.896 prec=0.932 rec=0.955

In [301]:
# =========================================
# Bloc 10 — Pipeline global MLP (détection + classification)
# =========================================
X_test_global = np.vstack([X_clean_test_np, X_adv_test_np])
y_is_adv      = np.concatenate([np.zeros(len(X_clean_test_np), dtype=np.int64),
                                np.ones(len(X_adv_test_np),   dtype=np.int64)])
y_true_cls    = np.concatenate([y_clean_test_np, y_adv_test_np])

Xemb_global   = extract_mlp_embeddings(model, X_test_global)
Xemb_global_z = (Xemb_global - mu) / sigma

with torch.no_grad():
    det_logits = []
    for i in range(0, len(Xemb_global_z), 512):
        xb = torch.from_numpy(Xemb_global_z[i:i+512]).float().to(device)
        det_logits.append(detector_mlp(xb).cpu().numpy())
    det_logits = np.concatenate(det_logits, axis=0)

det_prob = 1.0 / (1.0 + np.exp(-det_logits))
det_pred = (det_prob > TAU).astype(int)  # 1=adversarial (rejet), 0=propre (accepté)

accepted_mask = (det_pred == 0)
X_accepted    = X_test_global[accepted_mask]
y_true_acc    = y_true_cls[accepted_mask]
y_is_adv_acc  = y_is_adv[accepted_mask]
yhat_acc, _   = predict_mlp_classes(model, X_accepted)

n_adv_total       = int((y_is_adv == 1).sum())
n_adv_blocked     = int(((y_is_adv == 1) & (det_pred == 1)).sum())
pct_adv_blocked   = 100.0 * n_adv_blocked / max(1, n_adv_total)

clean_acc_mask    = (y_is_adv_acc == 0)
n_clean_accepted  = int(clean_acc_mask.sum())
n_clean_correct   = int((yhat_acc[clean_acc_mask] == y_true_acc[clean_acc_mask]).sum()) if n_clean_accepted > 0 else 0
pct_clean_correct = 100.0 * n_clean_correct / max(1, n_clean_accepted)

n_clean_total     = int((y_is_adv == 0).sum())
n_clean_blocked   = int(((y_is_adv == 0) & (det_pred == 1)).sum())
pct_false_rejects = 100.0 * n_clean_blocked / max(1, n_clean_total)

print("\n🔎 Pipeline global MLP (TEST étendu)")
print(f"• % adversariales bloquées              : {pct_adv_blocked:.2f}% ({n_adv_blocked}/{n_adv_total})")
print(f"• % données propres correctement classées : {pct_clean_correct:.2f}% ({n_clean_correct}/{max(1,n_clean_accepted)})")
print(f"• % faux rejets (propres bloquées)         : {pct_false_rejects:.2f}% ({n_clean_blocked}/{n_clean_total})")

print("\n📌 Breakdown par type d'attaque (TEST adversarial):")
start_adv = len(X_clean_test_np)
det_pred_adv = det_pred[start_adv:]
for tag in np.unique(src_test):
    m = (src_test == tag)
    n_tot = int(m.sum())
    n_blk = int((det_pred_adv[m] == 1).sum())
    print(f"- {tag:>10s}: bloquées {n_blk}/{n_tot} ({100.0*n_blk/max(1,n_tot):.1f}%)")


🔎 Pipeline global MLP (TEST étendu)
• % adversariales bloquées              : 62.96% (969/1539)
• % données propres correctement classées : 99.29% (140/141)
• % faux rejets (propres bloquées)         : 17.54% (30/171)

📌 Breakdown par type d'attaque (TEST adversarial):
- BIM@0.10000: bloquées 65/171 (38.0%)
- BIM@0.20000: bloquées 110/171 (64.3%)
-    CW@0.10: bloquées 156/171 (91.2%)
-    CW@0.30: bloquées 159/171 (93.0%)
- FGSM@0.10000: bloquées 64/171 (37.4%)
- FGSM@0.20000: bloquées 109/171 (63.7%)
- FGSM@0.30000: bloquées 130/171 (76.0%)
- PGD@0.10000: bloquées 65/171 (38.0%)
- PGD@0.20000: bloquées 111/171 (64.9%)
